In [45]:
from typing import Optional, Tuple, Text
import re

prefix='/pi/ws/agent_dispatcher/fixtures/'
DEFAULT_ENCODING = "utf-8"
filename='describes.md'

comment_regex = re.compile(r"<!--[\s\S]*?--!*>", re.MULTILINE)
item_regex = re.compile(r"\s*[-*+]\s*(.+)")

def strip_comments(text: Text) -> Text:
    """ Removes comments defined by `comment_regex` from `text`. """
    return re.sub(comment_regex, "", text)

def find_section_header(line: Text) -> Optional[Tuple[Text, Text]]:
    """Checks if the current line contains a section header
    and returns the section and the title."""
    # match = re.search(r"##\s*(.+?):(.+)", line)
    match = re.search(r"##\s*(.+)", line)
    if match is not None:
        return match.group(1)

    return None

def parse_item(line):
    print('..', line)

with open(prefix+filename, encoding=DEFAULT_ENCODING) as f:
    cnt=f.read()
    # print(cnt)
    s = strip_comments(cnt)
    current_section= ''
    for line in s.splitlines():
        line = line.strip()
        header = find_section_header(line)
        if header:
            current_section=header
            print(f"~ {current_section}")
        else:
            parse_item(line)
            # load_files(line)

~ loc_states
.. We are here.
.. They are all here.
.. I am here.
.. He is here and she is here.
.. You are here.
.. My family is here.
.. We are at school.
.. 
.. John is from London.
.. London is in Great Britain.
.. Maria is from Madrid.
.. Madrid is in Spain.
.. Peter and Martha are from Berlin.
.. Berlin is in Germany.
.. 
.. France is in Europe.
.. Egypt is in Africa.
.. Japan is in Asia.
.. Canada is in North America.
.. Panama is in Central America.
.. Brazil is in South America.
.. 
.. London is a capital city.
.. Madrid and Berlin are also capital cities.
.. Capital cities are big and noisy.
.. 
~ object_states
.. The family is not small.
.. The family is big.
.. He speaks English.
.. She speaks Spanish.
.. 
~ object_explain
.. We are a family.
.. Here is my address.
.. Those are the school children.
.. That is the teacher.
.. That is the class.
.. Learning languages is interesting.
.. 
~ behave
.. We are having class / a lesson.
.. I learn English.
.. You learn Spanish.
.. He

In [41]:
rule_pkg="""package: common
lang: ja

rules_1: &defaults
    f1: subj('adj', ガ=kindof('artifact', 'n'))
    f2: subj('adj', ガ=kindof('object', 'n'))
rules_2: &targets
    animal: verb(behaveof('protect', 'v'), head_acl=kindof('animal', 'n'), nsubj_pass=matchins('yang'))
    
intents:
  describe_object: 
    rules:
        <<: *defaults
        <<: *targets
    examples:
      - 彼のパソコンは便利じゃない。
      
  describe_artifact:         
    rules:
        *defaults  
    examples:
      - 彼の。
"""

import yaml
yaml.safe_load(rule_pkg)

{'package': 'common',
 'lang': 'ja',
 'rules_1': {'f1': "subj('adj', ガ=kindof('artifact', 'n'))",
  'f2': "subj('adj', ガ=kindof('object', 'n'))"},
 'rules_2': {'animal': "verb(behaveof('protect', 'v'), head_acl=kindof('animal', 'n'), nsubj_pass=matchins('yang'))"},
 'intents': {'describe_object': {'rules': {'f1': "subj('adj', ガ=kindof('artifact', 'n'))",
    'f2': "subj('adj', ガ=kindof('object', 'n'))",
    'animal': "verb(behaveof('protect', 'v'), head_acl=kindof('animal', 'n'), nsubj_pass=matchins('yang'))"},
   'examples': ['彼のパソコンは便利じゃない。']},
  'describe_artifact': {'rules': {'f1': "subj('adj', ガ=kindof('artifact', 'n'))",
    'f2': "subj('adj', ガ=kindof('object', 'n'))"},
   'examples': ['彼の。']}}}

In [34]:
pkg=yaml.safe_load(rule_pkg)
print(pkg['intents'].keys())

dict_keys(['describe_object', 'describe_artifact'])


In [37]:
intents=pkg['intents']
for k,intent in intents.items():
    print(k)
    print(intent['rules'])

describe_object
{'f1': "subj('adj', ガ=kindof('artifact', 'n'))", 'f2': "subj('adj', ガ=kindof('object', 'n'))", 'f3': "subj('adj', ガ=kindof('object', 'a'))"}
describe_artifact
{'f1': "subj('adj', ガ=kindof('artifact', 'n'))", 'f2': "subj('adj', ガ=kindof('object', 'n'))"}


In [44]:
from sagas.tool.dynamic_rules import dynamic_rule
pkg=yaml.safe_load(rule_pkg)
intents=pkg['intents']
for k,rule in intents['describe_object']['rules'].items():
    print('***', k)
    data = {'lang': 'id', "sents": 'Gajah adalah hewan yang dilindungi.'}
    dynamic_rule(data, rule)

*** f1
verb_domains dilindungi dilindungi ['rel', 'lemma', 'word', 'stems', 'lang', 'sents']
[('nsubj:pass', '4', 'yang', 'yang', ['yang'], ['c_pron', 'x_s--']),
 ('head_acl', '3', 'hewan', 'hewan', ['hewan'], ['c_noun', 'x_nsd'])]
✖ (_none_) subj with pos is ('adj',): False, ガ is kind_of(artifact,n): False
*** f2
verb_domains dilindungi dilindungi ['rel', 'lemma', 'word', 'stems', 'lang', 'sents']
[('nsubj:pass', '4', 'yang', 'yang', ['yang'], ['c_pron', 'x_s--']),
 ('head_acl', '3', 'hewan', 'hewan', ['hewan'], ['c_noun', 'x_nsd'])]
✖ (_none_) subj with pos is ('adj',): False, ガ is kind_of(object,n): False
*** animal
verb_domains dilindungi dilindungi ['rel', 'lemma', 'word', 'stems', 'lang', 'sents']
[('nsubj:pass', '4', 'yang', 'yang', ['yang'], ['c_pron', 'x_s--']),
 ('head_acl', '3', 'hewan', 'hewan', ['hewan'], ['c_noun', 'x_nsd'])]
✔ (_none_) verb with pos is behave_of(protect,v): True, head_acl is kind_of(animal,n): True, nsubj:pass is ins_match(equals: yang): True
.. results 

{'behave_of/default/...': {'category': 'protect', 'pos': 'v'},
 'kind_of/default/...': {'category': 'animal', 'pos': 'n'}}